15/03/2025

In [1]:
import pandas as pd

In [2]:
df_clienti = pd.read_csv(r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\data\clienti.csv",sep = ";")


In [3]:
df_clienti.isna().sum()

IDCliente                0
RagSoc                   0
Nominativo            3319
Categoria                0
NominativoContatto    1697
Indirizzo               73
CAP                    152
Comune                  56
Prov                   180
Paese                    2
Zona                  1773
PIVA                   247
Cellulare1            2971
Persona1              3021
emailpers1            6192
Cellulare2            5517
Persona2              5413
emailpers2            6304
Cellulare3            6349
Persona3              6287
emailpers3            6471
cellulare4            6547
persona4              6522
emailpers4            6546
cellulare5            6601
persona5              6584
emailpers5            6593
RagSoc_Dest           4916
Indirizzo_Dest        4883
dtype: int64

### Full text creation

In [5]:
# Funzione per creare la colonna full_text
def create_full_text(row, prefix="search_document"):
    return f"{prefix}: " + ", ".join(
        f"{col}: {val}"
        for col, val in row.items() 
        if pd.notna(val)
    )


# Creazione della nuova colonna full_text
df_clienti["full_text"] = df_clienti.apply(create_full_text, axis=1)

In [7]:
df_clienti_full_text = df_clienti["full_text"]

In [8]:
df_clienti_full_text

0       search_document: IDCliente: 1184, RagSoc: CIRC...
1       search_document: IDCliente: 1185, RagSoc: CIMB...
2       search_document: IDCliente: 1187, RagSoc: CIRC...
3       search_document: IDCliente: 1190, RagSoc: CIRC...
4       search_document: IDCliente: 1193, RagSoc: CIRC...
                              ...                        
6615    search_document: IDCliente: 13599, RagSoc: DFM...
6616    search_document: IDCliente: 13600, RagSoc: OST...
6617    search_document: IDCliente: 13601, RagSoc: JOI...
6618    search_document: IDCliente: 13602, RagSoc: ACQ...
6619    search_document: IDCliente: 13603, RagSoc: CLI...
Name: full_text, Length: 6620, dtype: object

### Embedding generation

NB: sulla repo di nomic-embed-text dicono che bisogna mettere una chiave per dire che tipo di task si vuole fare.

In [9]:
import ollama
import json

In [11]:
def get_embedding(text):
    try:
        response = ollama.embeddings(model="nomic-embed-text", prompt=text)
        return response['embedding']
    except Exception as e:
        print(f"Errore durante l'embedding: {e}")
        return None

# Creazione della colonna con gli embeddings
df_clienti_emb = df_clienti_full_text.apply(get_embedding)

In [12]:
df_clienti_emb = pd.DataFrame(df_clienti_emb)
# Salvataggio su file Parquet
df_clienti_emb.to_parquet("embeddings_v2.parquet", engine="pyarrow")

print("Embeddings salvati")

Embeddings salvati


ci ha messo 13 min x 6600 clienti.

la dimensione è di 768

### OCR:

per pdf scritti

In [ ]:
# from langchain.document_loaders import PyPDFLoader

# # Percorso del file PDF
# pdf_path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\langchain-ollama-elasticsearch\data\press-physics-nobel-prize-2024.pdf"

# # Carica il PDF
# loader = PyPDFLoader(pdf_path)
# docs = loader.load()

# # Unisce tutto il testo in una sola stringa
# full_text = "\n".join([doc.page_content for doc in docs])

# # Stampa i primi 1000 caratteri per verifica
# print(full_text)  # Stampiamo solo una parte per evitare output troppo lungo


risultati pessimi

In [ ]:
# import fitz  # PyMuPDF
# from PIL import Image
# import pytesseract
# import io

# pdf_path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\data\inputs\A4M_15122024_006852 copy.pdf"

# # Apri il PDF
# doc = fitz.open(pdf_path)

# # Lista per salvare il testo OCR estratto
# ocr_text = []

# for page in doc:
#     pix = page.get_pixmap()
    
#     # Converti in immagine PIL
#     img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

#     # OCR con Tesseract
#     text = pytesseract.image_to_string(img, lang="eng")  # Usa "ita" se il testo è in italiano
#     ocr_text.append(text)

# # Unisci tutto il testo OCR
# full_text = "\n".join(ocr_text)

# # Stampa il risultato
# print(full_text)


con gemini flash

In [13]:
import google.generativeai as genai
import os
import base64
from dotenv import load_dotenv

# Carica la chiave API da variabili d'ambiente
load_dotenv()
API_KEY = os.environ.get("GOOGLE_API_KEY")

# Configura Gemini
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Percorso del PDF
doc_path = r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\data\inputs\bomaki.pdf"

# Leggi e codifica il file in Base64
with open(doc_path, "rb") as doc_file:
    doc_data = base64.standard_b64encode(doc_file.read()).decode("utf-8")

# Prompt per forzare solo la trascrizione
prompt = """
Trascrivi fedelmente il contenuto di questo documento PDF. Non aggiungere spiegazioni, riassunti o commenti.
Devi solo riportare il testo esattamente come appare nel documento.
"""

# Genera il contenuto
response = model.generate_content([
    {'mime_type': 'application/pdf', 'data': doc_data},
    prompt
])

# Stampa la trascrizione
print(response.text)


BOMAKI
URAMAKERIA NIPPO BRASILIANA
Ordine di
acquisto
Cod. Fornitore
F100143
Vs. Riferimento
Agente
Condizioni e modalità di pagamento
Bonifico SEPA 30 GG DF FM
Numero
230032104
P.IVA/Cod. fisc.
IT01576310039
Data
13/12/2024
Valuta
EUR
Pag.
1 of 1
101 Srl
Sede legale Corso Sempione 20154 Milano (MI) - Italia
P.IVA/C Fiscale IT04995690965
R.Ε.Α. 561011
Tel. 0289071703
Email: amministrazione@bomaki.it
Web: www.bomaki.it
Spett.le
ZOPPIS SRL
VIA PER ALZO 21
28017 SAN MAURIZIO D'OPAGLIO (NO) -
ITALY
Indirizzo di spedizione
Bomaki di:
Via Bernardino Corio 8
20135 MILANO
ITALYVia Bernardino Corio 8
20135 MILANO

Cod. articola Descrizione articolo Qta U.M. Prezzo netto Cod. IVA Imponibile
19675 ACQUA BRILLANTE TONICA VAP 1.00 CT 11.95 22.00 14.58
15061 CACHACA LEBLON 70 MAISON LEBLON 6.00 CT 603.00 22.00 735.66
8729 GIN HENDRICK'S CL100 2.00 PZ 65.90 22.00 80.40
15666 HIERBAS DE IBIZA Litro AROMATICAS DE IBIZA! 12.00 PZ 203.40 22.00 248.15
24626 SAMBUCA RAMAZZOTI CL70 3.00 PZ 20.85 22.00 25.44

### RAG

In [18]:
df_clienti_emb_v2 = pd.read_parquet(r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\notebooks\embeddings_v2.parquet")

In [19]:
df_clienti_emb_v1 = pd.read_parquet(r"C:\Users\enduser\Desktop\PythonPostLaurea\ZOPPIS\ZOPPIS\notebooks\embeddings.parquet")

In [22]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cosine

def cosine_similarity(vec1, vec2):
    """Calcola la similarità coseno tra due vettori di embedding."""
    return 1 - cosine(vec1, vec2)

# Supponiamo di avere due dataframe con la colonna "full_text" contenente i vettori di embedding
df1 = df_clienti_emb_v1
df2 = df_clienti_emb_v2

# Converto gli embedding in array numpy
embeddings1 = np.array(df1["full_text"].tolist())
embeddings2 = np.array(df2["full_text"].tolist())

# Calcolo la similarità coseno per ogni coppia di embeddings corrispondenti
cosine_similarities = np.array([
    cosine_similarity(emb1, emb2) for emb1, emb2 in zip(embeddings1, embeddings2)
])

# Aggiungo i risultati al dataframe per analisi
df_similarity = pd.DataFrame({
    "cosine_similarity": cosine_similarities
})

# Visualizza statistiche sulle similarità
print(df_similarity.describe())


       cosine_similarity
count        6620.000000
mean            0.917193
std             0.010179
min             0.881335
25%             0.910358
50%             0.917172
75%             0.924128
max             0.950927


Queste sono le differenze fra v1 (senza l'item "document search:") e v2 (con l'item "document search:")

In [14]:
ordine = response.text

In [16]:
query = "search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine: \n" + ordine

In [25]:
query_emb = get_embedding(query)

In [28]:
import numpy as np
from scipy.spatial.distance import cdist

In [30]:
# Simuliamo la Serie di embedding
embeddings_series = df_clienti_emb["full_text"]

# Embedding della query
query_embedding = query_emb  # Modifica con il tuo embedding


# Converti la Serie in un array NumPy con dtype=float
embedding_matrix = np.array(embeddings_series.tolist(), dtype=np.float32)

# Assicuriamoci che la dimensione sia corretta
print("Shape della matrice degli embedding:", embedding_matrix.shape)
print("Shape dell'embedding della query:", len(query_embedding))

# Calcola la similarità del coseno
similarities = 1 - cdist([query_embedding], embedding_matrix, metric="cosine")[0]

# Ottieni i top 5 indici ordinati per similarità
top_5_indices = np.argsort(similarities)[::-1][:5]

# Mostra i risultati
print("Top 5 Indici più simili:", top_5_indices)
print("Similarità corrispondenti:", similarities[top_5_indices])

Shape della matrice degli embedding: (6620, 768)
Shape dell'embedding della query: 768
Top 5 Indici più simili: [4128 5703 4103 3830 3829]
Similarità corrispondenti: [0.78400688 0.7827887  0.77768747 0.77200334 0.7709059 ]


In [37]:
top_5_indices

array([4128, 5703, 4103, 3830, 3829], dtype=int64)

In [54]:
len(df_clienti_full_text.loc[top_5_indices].to_list()[0])

401

In [56]:
df_clienti_full_text

0       search_document: IDCliente: 1184, RagSoc: CIRC...
1       search_document: IDCliente: 1185, RagSoc: CIMB...
2       search_document: IDCliente: 1187, RagSoc: CIRC...
3       search_document: IDCliente: 1190, RagSoc: CIRC...
4       search_document: IDCliente: 1193, RagSoc: CIRC...
                              ...                        
6615    search_document: IDCliente: 13599, RagSoc: DFM...
6616    search_document: IDCliente: 13600, RagSoc: OST...
6617    search_document: IDCliente: 13601, RagSoc: JOI...
6618    search_document: IDCliente: 13602, RagSoc: ACQ...
6619    search_document: IDCliente: 13603, RagSoc: CLI...
Name: full_text, Length: 6620, dtype: object

In [57]:
rag_results = "\n".join(df_clienti_full_text.loc[top_5_indices].astype(str).tolist())


In [58]:
rag_results

'search_document: IDCliente: 10044, RagSoc: IOI S.R.L. - BOMAKI MELZO, Nominativo: BOMAKI, Categoria: RISTORANTE, NominativoContatto: ROBERTA GIACOMINO, Indirizzo: CORSO SEMPIONE, 8, CAP: 20154, Comune: MILANO, Prov: MI, Paese: Italia, Zona: MI-EST, PIVA: 04995690965, Cellulare1: 347 0495915, Persona1: FRANCESCO, emailpers2: amministrazione@bomaki.it, RagSoc_Dest: IDEM, Indirizzo_Dest: VIA MELZO, 28\nsearch_document: IDCliente: 12518, RagSoc: IOI S.R.L. - BOMAKI TOR DI QUINTO, Nominativo: BOMAKI, Categoria: RISTORANTE, NominativoContatto: SIG. MARCO PERROTTA, Indirizzo: CORSO SEMPIONE,8, CAP: 20154, Comune: MILANO, Prov: MI, Paese: Italia, Zona: ROMA, PIVA: 04995690965, Cellulare1: 346 8443085, Persona1: MARCO PERROTTA, Cellulare2: 366 6514832, Persona2: ROBERTA GIACOMINO, RagSoc_Dest: IOI S.R.L. - BOMAKI TOR DI QUINTO, Indirizzo_Dest: VIALE TOR DI QUINTO,35\nsearch_document: IDCliente: 10009, RagSoc: IOI S.R.L. - BOMAKI CITY LIFE, Categoria: RISTORANTE, NominativoContatto: FRANCESCO, 

In [51]:
len(df_clienti_full_text.loc[top_5_indices].to_string())

289

In [60]:
rag_results

'search_document: IDCliente: 10044, RagSoc: IOI S.R.L. - BOMAKI MELZO, Nominativo: BOMAKI, Categoria: RISTORANTE, NominativoContatto: ROBERTA GIACOMINO, Indirizzo: CORSO SEMPIONE, 8, CAP: 20154, Comune: MILANO, Prov: MI, Paese: Italia, Zona: MI-EST, PIVA: 04995690965, Cellulare1: 347 0495915, Persona1: FRANCESCO, emailpers2: amministrazione@bomaki.it, RagSoc_Dest: IDEM, Indirizzo_Dest: VIA MELZO, 28\nsearch_document: IDCliente: 12518, RagSoc: IOI S.R.L. - BOMAKI TOR DI QUINTO, Nominativo: BOMAKI, Categoria: RISTORANTE, NominativoContatto: SIG. MARCO PERROTTA, Indirizzo: CORSO SEMPIONE,8, CAP: 20154, Comune: MILANO, Prov: MI, Paese: Italia, Zona: ROMA, PIVA: 04995690965, Cellulare1: 346 8443085, Persona1: MARCO PERROTTA, Cellulare2: 366 6514832, Persona2: ROBERTA GIACOMINO, RagSoc_Dest: IOI S.R.L. - BOMAKI TOR DI QUINTO, Indirizzo_Dest: VIALE TOR DI QUINTO,35\nsearch_document: IDCliente: 10009, RagSoc: IOI S.R.L. - BOMAKI CITY LIFE, Categoria: RISTORANTE, NominativoContatto: FRANCESCO, 

In [100]:
query

"search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine: \nBOMAKI\nURAMAKERIA NIPPO BRASILIANA\nOrdine di\nacquisto\nCod. Fornitore\nF100143\nVs. Riferimento\nAgente\nCondizioni e modalità di pagamento\nBonifico SEPA 30 GG DF FM\nNumero\n230032104\nP.IVA/Cod. fisc.\nIT01576310039\nData\n13/12/2024\nValuta\nEUR\nPag.\n1 of 1\n101 Srl\nSede legale Corso Sempione 20154 Milano (MI) - Italia\nP.IVA/C Fiscale IT04995690965\nR.Ε.Α. 561011\nTel. 0289071703\nEmail: amministrazione@bomaki.it\nWeb: www.bomaki.it\nSpett.le\nZOPPIS SRL\nVIA PER ALZO 21\n28017 SAN MAURIZIO D'OPAGLIO (NO) -\nITALY\nIndirizzo di spedizione\nBomaki di:\nVia Bernardino Corio 8\n20135 MILANO\nITALYVia Bernardino Corio 8\n20135 MILANO\n\nCod. articola Descrizione articolo Qta U.M. Prezzo netto Cod. IVA Imponibile\n19675 ACQUA BRILLANTE TONICA VAP 1.00 CT 11.95 22.00 14.58\n15061 CACHACA LEBLON 70 MAISON LEBLON 6.00 CT 603.00 22.00 735.66\n8729 GIN HENDRICK'S CL100 2.00 PZ 65.90 22.00 80.40\n

In [ ]:
rag_results = "\n".join(df_clienti_full_text.loc[top_5_indices].astype(str).tolist())
prompt_rag = f"""search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine:
🚀 **IMPORTANTE: INIZIO ORDINE** 🚀

\nBOMAKI\nURAMAKERIA NIPPO BRASILIANA\nOrdine di\nacquisto\nCod. Fornitore\nF100143\nVs. Riferimento\nAgente\nCondizioni e modalità di pagamento\nBonifico SEPA 30 GG DF FM\nNumero\n230032104\nP.IVA/Cod. fisc.\nIT01576310039\nData\n13/12/2024\nValuta\nEUR\nPag.\n1 of 1\n101 Srl\nSede legale Corso Sempione 20154 Milano (MI) - Italia\nP.IVA/C Fiscale IT04995690965\nR.Ε.Α. 561011\nTel. 0289071703\nEmail: amministrazione@bomaki.it\nWeb: www.bomaki.it\nSpett.le\nZOPPIS SRL\nVIA PER ALZO 21\n28017 SAN MAURIZIO D'OPAGLIO (NO) -\nITALY\nIndirizzo di spedizione\nBomaki di:\nVia Bernardino Corio 8\n20135 MILANO\nITALYVia Bernardino Corio 8\n20135 MILANO\n\nCod. articola Descrizione articolo Qta U.M. Prezzo netto Cod. IVA Imponibile\n19675 ACQUA BRILLANTE TONICA VAP 1.00 CT 11.95 22.00 14.58\n15061 CACHACA LEBLON 70 MAISON LEBLON 6.00 CT 603.00 22.00 735.66\n8729 GIN HENDRICK'S CL100 2.00 PZ 65.90 22.00 80.40\n15666 HIERBAS DE IBIZA Litro AROMATICAS DE IBIZA! 12.00 PZ 203.40 22.00 248.15\n24626 SAMBUCA RAMAZZOTI CL70 3.00 PZ 20.85 22.00 25.44\n24945 SHARIS FELLUGA VENEZIE 30.00 PZ 289.50 22.00 353.19\n13035 TEQUILA PATRON Silver CL70 2.00 PZ 67.50 22.00 82.35\n20577 VODKA 42 BELOW LITRO 30.00 PZ 436.50 22.00 532.53\n

 **IMPORTANTE: FINE ORDINE**

 "sapendo che tramite RAG ho ottenuto i full text (stringa contenente tutti i dati strutturati tabellari) dei clienti più simili all'ordine in oggetto. Riporto di seguito i dati full text dei clienti più simili all'ordine. Dai importanza sopratutto alla destinazione dell'ordine e alla ragione sociale dell'ordine (l'Indirizzo Dest la RagSoc nella tabella) per scegliere la riga trai top-k che ti propongo! \n"
  
 🚀 **IMPORTANTE: INIZIO ELENCO DATI ESTRATTI TRAMITE RAG** 🚀
 {rag_results}

  **IMPORTANTE: FINE ELENCO DATI ESTRATTI TRAMITE RAG**

"\nRestituiscimi soltanto l'IDCliente, la RagSoc e l'Indirizzo Dest"" 
"""

In [104]:
prompt_rag

'search_query: Trovami l\'IDCliente (Codice Identificatore Cliente) del seguente ordine:\n🚀 **IMPORTANTE: INIZIO ORDINE** 🚀\n\n\nBOMAKI\nURAMAKERIA NIPPO BRASILIANA\nOrdine di\nacquisto\nCod. Fornitore\nF100143\nVs. Riferimento\nAgente\nCondizioni e modalità di pagamento\nBonifico SEPA 30 GG DF FM\nNumero\n230032104\nP.IVA/Cod. fisc.\nIT01576310039\nData\n13/12/2024\nValuta\nEUR\nPag.\n1 of 1\n101 Srl\nSede legale Corso Sempione 20154 Milano (MI) - Italia\nP.IVA/C Fiscale IT04995690965\nR.Ε.Α. 561011\nTel. 0289071703\nEmail: amministrazione@bomaki.it\nWeb: www.bomaki.it\nSpett.le\nZOPPIS SRL\nVIA PER ALZO 21\n28017 SAN MAURIZIO D\'OPAGLIO (NO) -\nITALY\nIndirizzo di spedizione\nBomaki di:\nVia Bernardino Corio 8\n20135 MILANO\nITALYVia Bernardino Corio 8\n20135 MILANO\n\nCod. articola Descrizione articolo Qta U.M. Prezzo netto Cod. IVA Imponibile\n19675 ACQUA BRILLANTE TONICA VAP 1.00 CT 11.95 22.00 14.58\n15061 CACHACA LEBLON 70 MAISON LEBLON 6.00 CT 603.00 22.00 735.66\n8729 GIN HEND

In [ ]:
rag_results = "\n".join(df_clienti_full_text.loc[top_5_indices].astype(str).tolist())
prompt_rag = query + "sapendo che tramite RAG ho ottenuto i full text (stringa contenente tutti i dati strutturati tabellari) dei clienti più simili all'ordine in oggetto. Riporto di seguito i dati full text dei clienti più simili all'ordine. Dai importanza sopratutto alla destinazione dell'ordine e alla ragione sociale dell'ordine (l'Indirizzo Dest la RagSoc nella tabella) per scegliere la riga trai top-k che ti propongo! \n" + rag_results + "\nRestituiscimi soltanto l'IDCliente, la RagSoc e l'Indirizzo Dest"

In [93]:
rag_results = "\n".join(df_clienti_full_text.loc[top_5_indices].astype(str).tolist())
prompt_rag = query + "sapendo che tramite RAG ho ottenuto i full text (stringa contenente tutti i dati strutturati tabellari) dei clienti più simili all'ordine in oggetto. Riporto di seguito i dati full text dei clienti più simili all'ordine. GUARDA LA destinazione dell'ordine (la via e in base a quello scegli la riga giusta) e alla ragione sociale dell'ordine (l'Indirizzo Dest la RagSoc nella tabella) per scegliere la riga trai top-k che ti propongo! \n" + rag_results + "\nRestituiscimi soltanto l'IDCliente, la RagSoc e l'Indirizzo Dest. SPiegami poi il perchè della tua scelta motivando su quali elementi dell'ordine ti sei affidato"

In [105]:
print(prompt_rag)

search_query: Trovami l'IDCliente (Codice Identificatore Cliente) del seguente ordine:
🚀 **IMPORTANTE: INIZIO ORDINE** 🚀


BOMAKI
URAMAKERIA NIPPO BRASILIANA
Ordine di
acquisto
Cod. Fornitore
F100143
Vs. Riferimento
Agente
Condizioni e modalità di pagamento
Bonifico SEPA 30 GG DF FM
Numero
230032104
P.IVA/Cod. fisc.
IT01576310039
Data
13/12/2024
Valuta
EUR
Pag.
1 of 1
101 Srl
Sede legale Corso Sempione 20154 Milano (MI) - Italia
P.IVA/C Fiscale IT04995690965
R.Ε.Α. 561011
Tel. 0289071703
Email: amministrazione@bomaki.it
Web: www.bomaki.it
Spett.le
ZOPPIS SRL
VIA PER ALZO 21
28017 SAN MAURIZIO D'OPAGLIO (NO) -
ITALY
Indirizzo di spedizione
Bomaki di:
Via Bernardino Corio 8
20135 MILANO
ITALYVia Bernardino Corio 8
20135 MILANO

Cod. articola Descrizione articolo Qta U.M. Prezzo netto Cod. IVA Imponibile
19675 ACQUA BRILLANTE TONICA VAP 1.00 CT 11.95 22.00 14.58
15061 CACHACA LEBLON 70 MAISON LEBLON 6.00 CT 603.00 22.00 735.66
8729 GIN HENDRICK'S CL100 2.00 PZ 65.90 22.00 80.40
15666 HIER

In [91]:
len(prompt_rag)

3792

In [106]:
response = ollama.chat(model="gemma3:4b", messages=[{"role": "user", "content": prompt_rag}])

print(response["message"]["content"])


10044


In [107]:
for chunk in ollama.chat(model="deepseek-r1:7b", messages=[{"role": "user", "content": prompt_rag}], stream=True):
    print(chunk["message"]["content"], end="", flush=True)


<think>
Alright, I need to help the user find the IDCliente (Codice Identificatore Cliente) for a specific order. Let me break down what they've provided.

First, there's an order with several details: product codes, quantities, unit prices, and some payment information. The order is from BOMAKI, and it seems to be related to a restaurant, given the terms like "RISORANTA" which translates to restaurant in Italian.

The user also included results from RAG (which I assume is a tool or method they use to extract customer data), providing several customer entries with IDs like 10044, 12518, etc. Each entry has details like RagSoc (a company name), contact person, and destination address.

The user's request is to return only the IDCliente, RagSoc, and Indirizzo Dest. From the RAG results provided:

- IDCliente: 10044, RagSoc: IOI S.R.L. - BOMAKI TOR DI QUINTO, Indirizzo_Dest: VIALE TOR DI QUINTO,35
- IDCliente: 12518, RagSoc: IOI S.R.L. - BOMAKI TOR DI QUINTO, Indirizzo_Dest: VIALE TOR DI 

In [97]:
!ollama list

NAME                       ID              SIZE      MODIFIED     
nomic-embed-text:latest    0a109f422b47    274 MB    25 hours ago    
gemma3:1b                  2d27a774bc62    815 MB    2 days ago      
llava-phi3:3.8b            c7edd7b87593    2.9 GB    2 days ago      
gemma3:4b                  c0494fe00251    3.3 GB    2 days ago      
deepseek-r1:7b             0a8c26691023    4.7 GB    2 days ago      
gemma3:12b                 6fd036cefda5    8.1 GB    2 days ago      
llama3.2:3b                a80c4f17acd5    2.0 GB    6 days ago      
llama3.2:latest            a80c4f17acd5    2.0 GB    2 months ago    


In [108]:
def count_tokens_simple(text):
    return len(text.split())

print(count_tokens_simple(prompt_rag))


529


In [111]:
prompt_rag

'search_query: Trovami l\'IDCliente (Codice Identificatore Cliente) del seguente ordine:\n🚀 **IMPORTANTE: INIZIO ORDINE** 🚀\n\n\nBOMAKI\nURAMAKERIA NIPPO BRASILIANA\nOrdine di\nacquisto\nCod. Fornitore\nF100143\nVs. Riferimento\nAgente\nCondizioni e modalità di pagamento\nBonifico SEPA 30 GG DF FM\nNumero\n230032104\nP.IVA/Cod. fisc.\nIT01576310039\nData\n13/12/2024\nValuta\nEUR\nPag.\n1 of 1\n101 Srl\nSede legale Corso Sempione 20154 Milano (MI) - Italia\nP.IVA/C Fiscale IT04995690965\nR.Ε.Α. 561011\nTel. 0289071703\nEmail: amministrazione@bomaki.it\nWeb: www.bomaki.it\nSpett.le\nZOPPIS SRL\nVIA PER ALZO 21\n28017 SAN MAURIZIO D\'OPAGLIO (NO) -\nITALY\nIndirizzo di spedizione\nBomaki di:\nVia Bernardino Corio 8\n20135 MILANO\nITALYVia Bernardino Corio 8\n20135 MILANO\n\nCod. articola Descrizione articolo Qta U.M. Prezzo netto Cod. IVA Imponibile\n19675 ACQUA BRILLANTE TONICA VAP 1.00 CT 11.95 22.00 14.58\n15061 CACHACA LEBLON 70 MAISON LEBLON 6.00 CT 603.00 22.00 735.66\n8729 GIN HEND

In [112]:
import google.generativeai as genai
import os
import base64
from dotenv import load_dotenv

# Carica la chiave API da variabili d'ambiente
load_dotenv()
API_KEY = os.environ.get("GOOGLE_API_KEY")

# Configura Gemini
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content(prompt_rag)
print(response.text)


IDCliente: 9550, RagSoc: IOI SRL - BOMAKI PORTA ROMANA, Indirizzo Dest: VIA BERNARDINO CORIO, 8



### verifiche

In [34]:
df_clienti_full_text.loc[[4128, 5703, 4103, 3830, 3829]]

4128    search_document: IDCliente: 10044, RagSoc: IOI...
5703    search_document: IDCliente: 12518, RagSoc: IOI...
4103    search_document: IDCliente: 10009, RagSoc: IOI...
3830    search_document: IDCliente: 9551, RagSoc: IOI ...
3829    search_document: IDCliente: 9550, RagSoc: IOI ...
Name: full_text, dtype: object

In [130]:
df_clienti_full_text.loc[[4128,4103,5703,5569,4104]]

4128    IDCliente: 10044, RagSoc: IOI S.R.L. - BOMAKI ...
4103    IDCliente: 10009, RagSoc: IOI S.R.L. - BOMAKI ...
5703    IDCliente: 12518, RagSoc: IOI S.R.L. - BOMAKI ...
5569    IDCliente: 12312, RagSoc: IOI S.R.L. - BOMAKI ...
4104    IDCliente: 10010, RagSoc: IOI S.R.L. - BOMAKI ...
Name: full_text, dtype: object

In [35]:
df_clienti.loc[[4128, 5703, 4103, 3830, 3829]]

,IDCliente,RagSoc,Nominativo,Categoria,NominativoContatto,Indirizzo,CAP,Comune,Prov,Paese,...,emailpers3,cellulare4,persona4,emailpers4,cellulare5,persona5,emailpers5,RagSoc_Dest,Indirizzo_Dest,full_text
4128,10044,IOI S.R.L. - BOMAKI MELZO,BOMAKI,RISTORANTE,ROBERTA GIACOMINO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"VIA MELZO, 28","search_document: IDCliente: 10044, RagSoc: IOI..."
5703,12518,IOI S.R.L. - BOMAKI TOR DI QUINTO,BOMAKI,RISTORANTE,SIG. MARCO PERROTTA,"CORSO SEMPIONE,8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IOI S.R.L. - BOMAKI TOR DI QUINTO,"VIALE TOR DI QUINTO,35","search_document: IDCliente: 12518, RagSoc: IOI..."
4103,10009,IOI S.R.L. - BOMAKI CITY LIFE,NaN,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"P.ZZA TRE TORRI, 1/L","search_document: IDCliente: 10009, RagSoc: IOI..."
3830,9551,IOI SRL - BOMAKI FOPPA,BOMAKI,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IOI SRL - BOMAKI FOPPA,"LARGO LA FOPPA,1","search_document: IDCliente: 9551, RagSoc: IOI ..."
3829,9550,IOI SRL - BOMAKI PORTA ROMANA,BOMAKI,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"VIA BERNARDINO CORIO, 8","search_document: IDCliente: 9550, RagSoc: IOI ..."


In [131]:
df_clienti.loc[[4128,4103,5703,5569,4104]]

,IDCliente,RagSoc,Nominativo,Categoria,NominativoContatto,Indirizzo,CAP,Comune,Prov,Paese,...,emailpers3,cellulare4,persona4,emailpers4,cellulare5,persona5,emailpers5,RagSoc_Dest,Indirizzo_Dest,full_text
4128,10044,IOI S.R.L. - BOMAKI MELZO,BOMAKI,RISTORANTE,ROBERTA GIACOMINO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"VIA MELZO, 28","IDCliente: 10044, RagSoc: IOI S.R.L. - BOMAKI ..."
4103,10009,IOI S.R.L. - BOMAKI CITY LIFE,NaN,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"P.ZZA TRE TORRI, 1/L","IDCliente: 10009, RagSoc: IOI S.R.L. - BOMAKI ..."
5703,12518,IOI S.R.L. - BOMAKI TOR DI QUINTO,BOMAKI,RISTORANTE,SIG. MARCO PERROTTA,"CORSO SEMPIONE,8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IOI S.R.L. - BOMAKI TOR DI QUINTO,"VIALE TOR DI QUINTO,35","IDCliente: 12518, RagSoc: IOI S.R.L. - BOMAKI ..."
5569,12312,IOI S.R.L. - BOMAKI ROMA PROPERZIO,BOMAKI,RISTORANTE,ROBERTA GIACOMINO,"CORSO SEMPIONE,8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IOI S.R.L. -BOMAKI ROMA PROPERZIO,"VIA PROPERZIO,20","IDCliente: 12312, RagSoc: IOI S.R.L. - BOMAKI ..."
4104,10010,IOI S.R.L. - BOMAKI SEMPIONE,BOMAKI,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"CORSO SEMPIONE, 10","IDCliente: 10010, RagSoc: IOI S.R.L. - BOMAKI ..."


In [ ]:
df_clienti[df_clienti["CAP"]==20135]

,IDCliente,RagSoc,Nominativo,Categoria,NominativoContatto,Indirizzo,CAP,Comune,Prov,Paese,...,emailpers3,cellulare4,persona4,emailpers4,cellulare5,persona5,emailpers5,RagSoc_Dest,Indirizzo_Dest,full_text


In [ ]:
f_filtered = df_clienti[df_clienti["RagSoc"].str.contains(r"\bBOMAKI\b", case=False, na=False, regex=True)]


In [ ]:
f_filtered

,IDCliente,RagSoc,Nominativo,Categoria,NominativoContatto,Indirizzo,CAP,Comune,Prov,Paese,...,emailpers3,cellulare4,persona4,emailpers4,cellulare5,persona5,emailpers5,RagSoc_Dest,Indirizzo_Dest,full_text
3828,9549,IOI SRL - BOMAKI SANZIO,BOMAKI,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"VIA RAFFAELLO SANZIO, 24","IDCliente: 9549, RagSoc: IOI SRL - BOMAKI SANZ..."
3829,9550,IOI SRL - BOMAKI PORTA ROMANA,BOMAKI,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"VIA BERNARDINO CORIO, 8","IDCliente: 9550, RagSoc: IOI SRL - BOMAKI PORT..."
3830,9551,IOI SRL - BOMAKI FOPPA,BOMAKI,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IOI SRL - BOMAKI FOPPA,"LARGO LA FOPPA,1","IDCliente: 9551, RagSoc: IOI SRL - BOMAKI FOPP..."
4103,10009,IOI S.R.L. - BOMAKI CITY LIFE,NaN,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"P.ZZA TRE TORRI, 1/L","IDCliente: 10009, RagSoc: IOI S.R.L. - BOMAKI ..."
4104,10010,IOI S.R.L. - BOMAKI SEMPIONE,BOMAKI,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"CORSO SEMPIONE, 10","IDCliente: 10010, RagSoc: IOI S.R.L. - BOMAKI ..."
4128,10044,IOI S.R.L. - BOMAKI MELZO,BOMAKI,RISTORANTE,ROBERTA GIACOMINO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IDEM,"VIA MELZO, 28","IDCliente: 10044, RagSoc: IOI S.R.L. - BOMAKI ..."
4555,10841,IOI S.R.L. - BOMAKI TORINO MURAZZI,BOMAKI,RISTORANTE,ROBERTA GIACOMINO,"CORSO SEMPIONE,8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IOI S.R.L. BOMAKI TORINO MURAZZI,"VIA MURAZZI DEL PO,29 SUL FIUME DEL PO","IDCliente: 10841, RagSoc: IOI S.R.L. - BOMAKI ..."
4780,11181,IOI S.R.L. - BOMAKI BATTISTI TORINO,BOMAKI,RISTORANTE,ROBERTA GIACOMINO,"CORSO SEMPIONE,8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IOI S.R.L. - BOMAKI BATTISTI TORINO,"VIA CESARE BATTISTI,3 E","IDCliente: 11181, RagSoc: IOI S.R.L. - BOMAKI ..."
4923,11397,IOI S.R.L. - BOMAKI GARIBALDI,BOMAKI,RISTORANTE,ROBERTA GIACOMINO,"CORSO SEMPIONE,8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IOI S.R.L. - BOMAKI GARIBALDI,"CORSO GARIBALDI ,79","IDCliente: 11397, RagSoc: IOI S.R.L. - BOMAKI ..."
5490,12205,IOI SRL - BOMAKI NAVIGLIO,BOMAKI,RISTORANTE,FRANCESCO,"CORSO SEMPIONE, 8",20154,MILANO,MI,Italia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IOI SRL - BOMAKI NAVIGLIO,"VIA ALZAIA NAVIGLIO GRANDE, 36","IDCliente: 12205, RagSoc: IOI SRL - BOMAKI NAV..."
